In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args


In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.layers import Reshape, Input, Dense, Activation,Dropout,Flatten,BatchNormalization
from tensorflow.keras.models import Model,Sequential

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
import xgboost as xgb
import numpy as np
from keras.layers import BatchNormalization,GaussianNoise,Dense,Dropout
from keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.metrics import explained_variance_score
from sklearn.decomposition import PCA
import pickle
import statistics as stat

In [ ]:
TRAINING=False

In [ ]:
train = pd.read_csv('../input/jane-street-market-prediction/train.csv')
train = train.query('date > 85').reset_index(drop = True) 
train = train.astype({c: np.float32 for c in train.select_dtypes(include='float64').columns}) #limit memory use
train.fillna(train.mean(),inplace=True)
train = train.query('weight > 0').reset_index(drop = True)
#train['action'] = (train['resp'] > 0).astype('int')
train['action'] =  (  (train['resp_1'] > 0 ) & (train['resp_2'] > 0 ) & (train['resp_3'] > 0 ) & (train['resp_4'] > 0 ) &  (train['resp'] > 0 )   ).astype('int')
features = [c for c in train.columns if 'feature' in c]

resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

In [ ]:
f_mean = np.mean(train[features[1:]].values,axis=0)


In [ ]:
def load_standardizer():
        with open('../input/jane-street-tcn-bottleneck-auto-encoder/Standardizer.pkl', 'rb') as pickle_file:
             return pickle.load(pickle_file)

In [ ]:
Standardizer = load_standardizer()

In [ ]:
def load_model(name:str):
    return tf.keras.models.load_model(name)

In [ ]:
neural_network = load_model('../input/jane-street-tcn-bottleneck-auto-encoder/nn-ae.h5')

In [ ]:
encoder= load_model('../input/jane-street-tcn-bottleneck-auto-encoder/autoencoder.h5')

In [ ]:
temporal_neural_net= load_model('../input/jane-street-tcn-bottleneck-auto-encoder/tcn-ae.h5')

In [ ]:
encoder.summary()

In [ ]:
from tqdm import tqdm
from random import choices
import random


import kerastuner as kt

In [ ]:
models = [temporal_neural_net,neural_network]

In [ ]:
len(f_mean)


In [ ]:
if not TRAINING:
    import janestreet
    #janestreet.competition.make_env.__called__ = False
    env = janestreet.make_env()
    th = 0.5
    #w = np.asarray([0.1,0.1,0.1,0.5,0.2])
    
    for (test_df, pred_df) in tqdm(env.iter_test()):
        if test_df['weight'].item() > 0:
            x_tt = test_df.loc[:, features]
            if np.isnan(x_tt.iloc[:, 1:].values.sum()):
                
                x_tt.iloc[:, 1:] = np.nan_to_num(x_tt.iloc[:, 1:]) + np.isnan(x_tt.iloc[:, 1:]) * f_mean

            
            x_tt = x_tt.values
            x_tt = Standardizer.transform(x_tt)
            x_tt = encoder(x_tt).numpy()
            pred = np.mean([model(x_tt,training=False) for model in models],axis=0).squeeze()
            pred = np.mean(pred)
            pred_df.action = np.where(pred > th, 1, 0).astype(int)
        else:
            pred_df.action = 0
        env.predict(pred_df)